# Deploy a Grafana Dashboard
To track the different stocks on a live dashboard we will use **Grafana**.<br>
We will use [Grafwiz](https://github.com/v3io/grafwiz) to define and deploy the dashboard directly from this notebook

In [1]:
# install prerequsits 
# !pip install git+https://github.com/v3io/grafwiz --upgrade
# !pip install v3io_frames

In [2]:
import mlrun
project = mlrun.get_or_create_project(name='stocks',user_project=True, context="src/")

> 2022-08-08 14:38:06,653 [info] loaded project stocks from MLRun DB


In [3]:
import grafwiz
import os
import v3io_frames as v3f

framesd = os.getenv("V3IO_FRAMESD",'framesd:8081')
client = v3f.Client(framesd, container=os.getenv('V3IO_CONTAINER', 'projects'))

grafwiz.DataSource(name='iguazio').deploy('http://grafana',use_auth=True)

dash = grafwiz.Dashboard("stocks", start='now-7d', dataSource='iguazio')


# folder_name = os.listdir(path='/v3io/projects/stocks-'+ os.environ['V3IO_USERNAME'] + '/FeatureStore/news/nosql/sets/news')[0]
# news_kv = '/stocks-'+ os.environ['V3IO_USERNAME'] + '/FeatureStore/news/nosql/sets/news/'+ folder_name
# # client.execute("kv",table = news_kv, command = "infer")

# tbl = grafwiz.Table(title='Current news Value', span=12,columns=['ticker','sentiment']).source(table=news_kv,fields=['ticker','sentiment'],container='projects')
# dash.row([tbl])


df_columns = ['Close', 'High', 'Low', 'Open', 'Volume', 'sentiment',
              'ticker2onehot_A', 'ticker2onehot_AAL', 'ticker2onehot_AAP',
              'ticker2onehot_AAPL', 'tickers', 'datetime']

prediction_kv = '/stocks-'+ os.environ['V3IO_USERNAME'] + '/artifacts/stocks_prediction'
# client.execute("kv",table = prediction_kv, command = "infer")

log = grafwiz.Table(title='Stocks Prediction Values', span=12).source(table=prediction_kv,fields=df_columns,container='projects')
dash.row([log])

dash.deploy('http://grafana')

Datasource iguazio already exists
Datasource iguazio created successfully
Dashboard stocks created successfully


In [4]:
# framesd = os.getenv("V3IO_FRAMESD",'framesd:8081')
# client = v3f.Client(framesd, container=os.getenv('V3IO_CONTAINER', 'projects'))

# dash = grafwiz.Dashboard("stocks", start='now-7d', dataSource='iguazio')



# grafana_url = 'http://grafana'
# grafwiz.DataSource(name='iguazio').deploy(grafana_url,use_auth=True)

# prediction_kv = '/stocks-'+ os.environ['V3IO_USERNAME'] + '/artifacts/stocks_prediction'
# dash.template(name="SYMBOL", label="Symbol", query="fields=Close" + ";table=" + prediction_kv + ";backend=kv;container=projects")
# grafwiz.DataSource(name='stocks-prediction', frames_url=prediction_kv).deploy(grafana_url, use_auth=False, overwrite=False)

# log = grafwiz.Table('Articles Log', dataSource='stocks-prediction', span=12).source(container='projects',fields=df_columns)
# dash.row([log])

# dash.deploy(grafana_url)